# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成計數編碼以及搭配邏輯斯迴歸的預測 (In[4], Out[4], In[5], Out[5]) 
- 仿造範例, 完成雜湊編碼, 以及計數編碼+雜湊編碼 搭配邏輯斯迴歸的預測 (In[6], Out[6], In[7], Out[7]) 
- 試著回答上述執行結果的觀察

# 作業1
* 參考範例，將鐵達尼的艙位代碼( 'Cabin' )欄位使用特徵雜湊 / 標籤編碼 / 目標均值編碼三種轉換後，  
與其他類別型欄位一起預估生存機率

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = 'data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Object Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


# 作業2
* 承上題，三者比較效果何者最好?

In [3]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
train_X = df_temp[:train_num]
estimator = LogisticRegression(solver='lbfgs')
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.7789000729487724


,Name,Sex,Ticket,Cabin,Embarked
0,155,1,720,185,3
1,286,0,816,106,0
2,523,0,914,185,3
3,422,0,65,70,3
4,22,1,649,185,3


In [4]:
# 加上 'Cabin' 欄位的計數編碼
count_df = df.groupby(['Cabin'])['Name'].agg({'Cabin_count':'size'}).reset_index()
count_df

c:\users\pero\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


,Cabin,Cabin_count
0,A10,1
1,A11,1
2,A14,1
3,A16,1
4,A18,1
5,A19,1
6,A20,1
7,A21,1
8,A23,1
9,A24,1


In [5]:
df = pd.merge(df, count_df, on=['Cabin'], how='left')
count_df.sort_values(by=['Cabin_count'], ascending=False).head(10)

,Cabin,Cabin_count
185,None,1014
80,C23 C25 C27,6
184,G6,5
47,B57 B59 B63 B66,5
60,B96 B98,4
180,F2,4
117,D,4
79,C22 C26,4
181,F33,4
183,F4,4


In [6]:
df

,Name,Sex,Ticket,Cabin,Embarked,Cabin_count
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S,1014
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C,2
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S,1014
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S,2
4,"Allen, Mr. William Henry",male,373450,None,S,1014
5,"Moran, Mr. James",male,330877,None,Q,1014
6,"McCarthy, Mr. Timothy J",male,17463,E46,S,2
7,"Palsson, Master. Gosta Leonard",male,349909,None,S,1014
8,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,347742,None,S,1014
9,"Nasser, Mrs. Nicholas (Adele Achem)",female,237736,None,C,1014


In [25]:
# 'Cabin'計數編碼 + 邏輯斯迴歸
df_cabin = pd.DataFrame()
for c in df.columns:
    df_cabin[c] = LabelEncoder().fit_transform(df[c])
train_X = df_cabin[:train_num]
estimator = LogisticRegression(solver='liblinear')
    #用lbfgs會有fail convergence的warning, 改用liblinear
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_cabin.head()

0.7811600308817652


,Name,Sex,Ticket,Cabin,Embarked,Cabin_count
0,155,1,720,185,3,6
1,286,0,816,106,0,1
2,523,0,914,185,3,6
3,422,0,65,70,3,1
4,22,1,649,185,3,6


In [26]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_hash = pd.DataFrame()
for c in object_features:
    df_hash[c] = LabelEncoder().fit_transform(df[c])
# 這邊的雜湊編碼, 是直接將 'Ticket' 的名稱放入雜湊函數的輸出數值, 為了要確定是緊密(dense)特徵, 因此除以10後看餘數
# 這邊的 10 是隨機選擇, 不一定要用 10, 同學可以自由選擇購小的數字試看看. 基本上效果都不會太好
df_hash['Cabin_Hash'] = df['Cabin'].map(lambda x:hash(x) % 10)
train_X = df_hash[:train_num]
estimator = LogisticRegression(solver='liblinear')
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_hash.head()

0.7777828254403613


,Name,Sex,Ticket,Cabin,Embarked,Cabin_Hash
0,155,1,720,185,3,3
1,286,0,816,106,0,9
2,523,0,914,185,3,3
3,422,0,65,70,3,4
4,22,1,649,185,3,3


In [28]:
# 'Cabin'計數編碼 + 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_all = pd.DataFrame()
for c in object_features:
    df_all[c] = LabelEncoder().fit_transform(df[c])
df_all['Cabin_Hash'] = df['Cabin'].map(lambda x:hash(x) % 10)
df_all['Cabin_Count'] = df['Cabin_count']
train_X = df_all[:train_num]
estimator = LogisticRegression(solver='liblinear')
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_all.head()

0.7822710722476556


,Name,Sex,Ticket,Cabin,Embarked,Cabin_Hash,Cabin_Count
0,155,1,720,185,3,3,1014
1,286,0,816,106,0,9,2
2,523,0,914,185,3,3,1014
3,422,0,65,70,3,4,2
4,22,1,649,185,3,3,1014


## pd.groupby()
OReilly_Python_Datascience_Handbook p.166-171

### GroupBy採用的是split-apply-combine的運算過程，
split: 依照指定的key value分解和重組dataframe

apply: 在某個特定的分組中計算某個函數，通常是聚合/轉換/過濾等。

combine: 合併之前的運算結果並將其變成陣列輸出

In [10]:
df1=pd.DataFrame({'key':['A','B','C','A','B','C'],'data':range(6)},columns=['key','data'])
df1

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


In [11]:
temp=df1.groupby('key')
    #groupby回傳的結果可視為dataframe的view,並不執行實際的運算，直到聚合運算被設定為止。
    #此種作法稱為lazy evaluation
temp

In [12]:
#若要產生結果，則要套用一個聚合運算至該groupby物件上
#聚合運算的種類：OReilly_Python_Datascience_Handbook p.58 and p.166
temp.sum()

,data
key,
A,3
B,5
C,7


In [13]:
#使用行星資料集，OReilly_Python_Datascience_Handbook p.163
#此dataset包含到2014年被發現的行星資料
import seaborn as sns
planets=sns.load_dataset('planets')
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [14]:
x=planets.groupby('method')
x

In [15]:
y=planets.groupby('method')['orbital_period']
    #從原始的dataframe中以參考column的方式選擇了一個series的group
    #orbital_period：軌道周期
y

In [16]:
print(y.median())
    #由此可看出針對每一種發現的方法和軌道周期之間的關係
print(x.median())
    #可看出y是x的subset

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64
                               number  orbital_period   mass  distance    year
method                                                                        
Astrometry                        1.0      631.180000    NaN    17.875  2011.5
Eclipse Timing Variations         2.0     4343.500000  5.125   315.360  2010.0
Imaging                           1.0    27500.000000    NaN    40.395  2009.0
Microlensing                      1.0     3300.000000    NaN  3840.000  2010.0
Orbital Brightness Mod

In [17]:
#groupby object支援迭代
for (method, group) in planets.groupby('method'):
    print("{0:30s} shape={1}".format(method,group.shape))

Astrometry                     shape=(2, 6)
Eclipse Timing Variations      shape=(9, 6)
Imaging                        shape=(38, 6)
Microlensing                   shape=(23, 6)
Orbital Brightness Modulation  shape=(3, 6)
Pulsar Timing                  shape=(5, 6)
Pulsation Timing Variations    shape=(1, 6)
Radial Velocity                shape=(553, 6)
Transit                        shape=(397, 6)
Transit Timing Variations      shape=(4, 6)


### Aggregate, filter, transform, apply

In [18]:
rng=np.random.RandomState(0)
df2=pd.DataFrame({'key':['A','B','C','A','B','C'],'data1':range(6),'data2':rng.randint(0,10,6)},columns=['key','data1','data2'])
df2

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [19]:
#aggregate()
df2.groupby('key').aggregate(['min',np.median,max])
    #aggregate()中可輸入字串('min'), 函式，串列或字典，然後一次進行聚合運算。

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

In [20]:
df2.groupby('key').aggregate({'data1':min,'data2':'max'})
    #此字典為欄名和運算的對應

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


In [21]:
planets.groupby('year')['method'].describe()

,count,unique,top,freq
year,,,,
1989,1,1,Radial Velocity,1
1992,2,1,Pulsar Timing,2
1994,1,1,Pulsar Timing,1
1995,1,1,Radial Velocity,1
1996,6,1,Radial Velocity,6
1997,1,1,Radial Velocity,1
1998,5,1,Radial Velocity,5
1999,15,1,Radial Velocity,15
2000,16,1,Radial Velocity,16
